Con người chúng ta có thể đưa ra được những quyết định và dự đoán những điều sắp xảy ra khi nhìn vào bối cảnh của tổng thể. Ví dụ: bạn đang lái xe và bạn nghĩ rằng người đi bộ phía trước có khả năng chuẩn bị băng qua làn đường, và bạn sẽ quyết định lái xe ở khoảng cách vừa đủ để tránh họ.

Hơn nữa, trong thực tế có nhiều sự kiện chịu sự hạn chế về thời gian. Trong các câu chuyện như phim hoạt hình, các diễn biến tiếp theo sẽ dựa trên các bối cảnh phía trước đó.

Recurrent Neural Networks là một mạng Neural Network có tính đến việc xem xét các bối cảnh. Trong bài viết này, sẽ đưa cho các bạn cái nhìn tổng quan về:
1. Recurrent Neural Networks (RNN) là gì?
2. Phân loại bài toán RNN
3. Một số ứng dụng của RNN
    - Dịch máy 
    - Nhận dạng giọng nói
    - Tóm tắt hình ảnh
    - Tạo hình ảnh từ mô tả 
4. Mô hình RNN và các vấn đề 
5. Hướng khắc phục các vấn đề của mô hình RNN 

### 1. Recurrent Neural Networks là gì?

Recurrent Neural Networks (RNN) - một cách dịch đại khái sang tiếng Việt là mạng thần kinh hồi quy. Sở dĩ, nó được gọi là Recurrent (hồi quy) vì nó thực hiện cùng một tác vụ cho tất cả các phần tử của một chuỗi với đầu ra phụ thuộc vào cả các phép tính trước đó. Nói cách khác, RNN có khả năng ghi nhớ các thông tin được tính toán trước đó (như là việc nhìn vào các bối cảnh phía trước rồi đưa ra các hành động tiếp theo). RNN là mô hình mạng thần kinh được thiết kế để nhận dữ liệu tuần tự kiểu dữ liệu time series (chuỗi thời gian). 

Đầu tiên, trước khi tìm hiểu về RNN, chúng ta hãy cùng xem cách mạng Neural thông thường lan truyền như hình dưới đây:

<img src="images/1.neural_net.png" style="width:50%;height:50%;">

Với mạng Neural Network thông thường, các nút màu đỏ là giá trị đầu vào còn màu xanh lá cây là giá trị đầu ra. Ví dụ trong bài toán về phân loại hình ảnh dùng [Convolutional Neural Network](https://aivietnam.ai/courses/aisummer2019/lessons/convolutional-neural-network/): đầu vào sẽ là các pixel và đầu ra là các ảnh cần phân loại. Theo cách này, bạn có thể thiết thế một mạng Neural Network có độ dài đầu vào và đầu ra cố định.

Tuy nhiên, các dữ liệu trong thực tế thường không có chiều dài cố định. Ví dụ: khi bạn viết một Status trên Facebook, nó không cố định chiều dài số lượng từ mà bạn viết. Chính vì vậy mà mạng RNN được thiết kế để xử lý các dữ liệu có đầu vào và đầu ra thay đổi.

Về cơ bản, một mô hình đơn giản của RNN có dạng như sau:
<img src="images/2_rnn.png" style="width:70%;height:70%;">


Trên thực tế, mô hình RNN sẽ được biểu thị như phía bên trái của hình trên. Nhưng để dễ hình dung hơn, chúng ra `Unfold` (hay Unroll) khai triển mô hình ra như phía bên phải. Ví dụ, nếu bạn có 1 câu có 4 chữ "I love you 3000", thì mạng RNN sẽ triển khai ra gồm 4-layer Neural Network. Trong đó, mỗi Layer tương ứng với 1 từ. Lúc này, việc tính toán bên trong RNN sẽ xảy ra như sau:

- $x_t$ là đầu vào của bước $t$. Ví dụ: $x_2$ là một one-hot vector tương ứng với chữ thứ 3 là "you" (vị trí index được tính từ 0).


- $s_t$ là trạng thái ẩn ở bước thứ $t$. Nó là “memory (bộ nhớ)” của mạng. $s_t$ được tính toán dựa trên trạng thái ẩn trước đó và đầu vào của bước hiện tại $s_t=f(Ux_t + Ws_{t-1})$. Hàm $f$ thường là một hàm phi tuyến như tanh hoặc ReLU. Đối với bước đầu tiên, giá trị $s_{-1}$ được gán bằng `0`.


- $o_t$ là output (đầu ra) ở bước t. Ví dụ: nếu bạn muốn dự đoán từ tiếp theo trong câu thì $o_t$ là một vector xác suất các từ trong danh sách từ vựng. $o_t = \mathrm{softmax}(Vs_t)$.

<font color='blue'>
    
**Chú ý:** Có một vài điều các bạn cần lưu ý ở đây !

- Bạn có thể coi trạng thái ẩn $s_t$ là "bộ nhớ" của mạng. $s_t$ sẽ lưu trữ các thông tin đã được tính toán ở phía trước. Đầu ra ở bước $o_t$ chỉ được tính dựa trên bộ nhớ ở bước thứ $t$. Trong thực tế, mạng RNN không thể lưu trữ được quá nhiều thông tin, và một số mạng khác như LSTM, GRU với cơ chế Attention sẽ giúp khắc phục nhược điểm này. 
- Không giống như các mạng Neural Network khác, mạng RNN sử dụng bộ tham số $(U, V, W)$ trong tất cả các bước. Nó phản ánh rằng, chúng ta thực hiện cùng 1 nhiệm vụ ở tất cả các bước. Và điều này làm giảm đáng kể số lượng tham số cần học trong mô hình.

**Tóm tắt**:

<table style="width:30%"> 
    <tr>
    <td > Input </td> 
    <td > $x$ </td> 
    </tr> 
    <tr>
    <td > Hidden state </td> 
    <td > $h$ hoặc $s$ </td> 
    </tr> 
    <tr>
    <td > Output </td> 
    <td > $o$ hoặc $\hat{y}$ </td> 
    </tr> 
    <tr>
    <td > Tham số mô hình </td> 
    <td > $U, V, W$ </td> 
    </tr> 
</table>

### 2. Phân loại bài toán RNN 

Về cơ bản, các loại bài toán về RNN có thể phân loại như sau:

<img src="images/8_rnn_classification.png" style="width:70%;height:70%;">

Trong hình trên, mỗi hình chữ nhật biểu thị một vector, và các mũi đại diện cho các function (Ví dụ: matrix multiply). Các vector đầu vào màu đỏ ($x$) và màu xanh dương là đầu ra ($o$). Màu xanh lá cây biểu thị cho trạng thái ẩn ($s$).

Từ trái qua phải chúng ta có:

1. one-to-one: mô hình cơ bản có đầu vào và đầu ra cố định, còn được gọi là Vanilla RNN. 
   Ví dụ: bài toán phân loại hình ảnh.

2. one-to-many: Sequence output (bài toán có 1 đầu vào và nhiều đầu ra). 
   Ví dụ: từ 1 hình ảnh đầu vào sinh ra các mô tả về hình ảnh này - chi tiết hơn xem phần 3.3 bên dưới.

3. many-to-one: Sequence input (ngược lại với one-to-many, bài toán này có nhiều đầu vào và chỉ có 1 đầu ra duy nhất). 
   Ví dụ: Phân tích cảm xúc, đánh giá nhận xét. Trong một bài toán đơn giản xét các đánh giá trên Foody.vn thì input là các câu nhận xét và output là đánh giá ngon (1) hoặc không ngon (0).

4. many-to-many 1: Sequence input và sequence output. 
   Ví dụ: Machine Learning Transaltion (dịch máy - Google dịch).

5. many-to-many 2: Đồng bộ sequence input và output. 
   Ví dụ: Phân loại video mà trong đó muốn gắn nhãn cho từng khung hình của video.

### 3. Một vài ứng dụng của RNN 

RNN là một mô hình mạnh có thể xử lý linh hoạt và có nhiều ứng dụng.

#### 3.1 Dịch máy (Machine Translation)

Trong những năm gần đây, độ chính xác của Google Translate tăng lên đáng kể nhờ vào việc sử dụng Neural Network biến thể của RNN. Trong dịch máy, kích thước đầu vào và kích thước đầu ra không cố định, và bạn có thể dịch từ tiếng Việt sang bất cứ ngôn ngữ nào khác. 

Mô hình Neural Machine Translate dựa trên cơ chế Sequence to Sequence Model (Seq2seq) gồm 2 phần chính là Encoder (bộ mã hoá) và Decoder (bộ giải mã). Cả hai thành phần này đều được hình thành từ các mạng Neural Networks.

<img src="images/3_RNN_Machine_Translation.png" style="width:50%;height:50%;">

Đối với mô hình dịch máy của Google sử dụng biến thể của RNN là LSTM cùng với cơ chế Attention, Seq2Seq, minh hoạ như hình dưới đây: 

<img src="images/4_Google_NMT.png" style="width:70%;height:70%;">


#### 3.2 Nhận dạng giọng nói 

Cũng giống như Machine Translation, đầu vào của mô hình nhận dạng giọng nói có độ dài thay đổi. Hình ảnh sau đây là minh hoạ cho mô hình nhận dạng giọng nói sử dụng Deep Neural Network được nghiên cứu bởi trường đại học Stanford.

<img src="images/5_stanford_speech.png" style="width:70%;height:70%;">

Trong nghiên cứu trên, âm thanh được coi như là một hình ảnh, lớp tích chập (Conv) được sử dụng ở đầu vào và đầu ra là RNN. Để biết thêm chi tiết về Convolution và CNN, các bạn hãy xem lại bài viết về CNN trên AIVIETNAM.AI.

#### 3.3 Mô tả hình ảnh 

Cùng với CNN, RNN được sử dụng để tự động sinh ra mô tả trong ảnh chưa được dán nhãn. Hình ảnh đầu vào được xử lý bởi CNN và đầu ra được xử lý bởi RNN. 

<img src="images/6_generating_image_desciptions.png" style="width:70%;height:70%;">

Các bạn có thể xem thêm tại: https://cs.stanford.edu/people/karpathy/deepimagesent/

#### 3.4 Tạo hình ảnh từ mô tả 

Với một câu đầu vào, bạn có thể sử dụng RNN để tạo ra hình ảnh từ mô tả. Hình ảnh sau đây là một ví dụ minh hoạ:

<img src="images/7_Generating_Image.png" style="width:70%;height:70%;">


### 4. Mô hình RNN và các vấn đề 

#### 4.1 Nhắc lại về RNN 

Mô hình RNN: 
<img src="images/2_rnn.png" style="width:70%;height:70%;">

- $x_t$ là đầu vào của bước $t$. 

- $s_t$ là trạng thái ẩn ở bước thứ $t$:
    - $s_t$ được tính toán dựa trên trạng thái ẩn trước đó và đầu vào của bước hiện tại $s_t=f(Ux_t + Ws_{t-1})$. 
    - Hàm $f$ thường là một hàm phi tuyến như `tanh` hoặc `ReLU`. Đối với bước đầu tiên, giá trị $s_{-1}$ được gán bằng `0`.


- $o_t$ là output (đầu ra) ở bước t. $o_t = \mathrm{softmax}(Vs_t)$.

<table style="width:30%"> 
    <tr>
    <td > Input </td> 
    <td > $x$ </td> 
    </tr> 
    <tr>
    <td > Hidden state </td> 
    <td > $h$ hoặc $s$ </td> 
    </tr> 
    <tr>
    <td > Output </td> 
    <td > $o$ hoặc $\hat{y}$ </td> 
    </tr> 
    <tr>
    <td > Tham số mô hình </td> 
    <td > $U, V, W$ </td> 
    </tr> 
</table>

Giả sử chúng ta có một danh list các word vector đầu vào như sau: $x_1, x_2, ..., x_{t-1}, x_{t}, x_{t+1}, ..., x_{T}$

Tại thời điểm $t$: 

> $s_t=f(Ux_t + Ws_{t-1})$

> $o_t$ hoặc $\hat{y}_t = \mathrm{softmax}(Vs_t)$

> Cost function có dạng như sau: $J^{(t)}(\theta) = - \displaystyle \sum_{j=1}^{|V|}(y_{t,j} \log{{\hat{y}_{t,j}}})$

Trong cả khoảng thời gian $T$: $J(\theta) = - \dfrac{1}{T} \displaystyle\sum_{t=1}^{T} \displaystyle \sum_{j=1}^{|V|}(y_{t,j} \log{{\hat{y}_{t,j}}})$

#### 4.2 Vấn đề trong RNN  
##### 4.2.1 Nhắc lại về vấn đề Vanishing / Exploding Gradient trong Neural Network

Trong quá trình thực hiện thuật toán Backpropagation Algorithm (thuật toán lan truyền ngược) có thể gây ra 2 hiện tượng là: Vanishing (đạo hàm tiến về 0) và Exploding (gây ra quá lớn).

<img src="images/10_vanishing_exploding.png" style="width:50%;height:50%;">

Xem thêm chi tiết về vấn đề này trong bài giảng của thầy Andrew Ng [tại đây](https://www.youtube.com/watch?v=qhXZsFVxGKo)

##### 4.2.2 Vấn đề Vanishing Gradient

Trong mô hình RNN có 3 tham số $U, V, W$ nên chúng ta cần phải tính toán đạo hàm 3 tham số này:

> $\frac{\partial J}{\partial U}$, $\frac{\partial J}{\partial V}$, $\frac{\partial J}{\partial W}$

 <img src="images/9_vanishing.png" style="width:50%;height:50%;">
 
 Thực hiện tính đạo hàm theo $W$ ta có:

> $\frac{\partial J_n}{\partial W} = \displaystyle \sum_{k=0}^n \frac{\partial J_n}{\partial \hat{y}_n} \frac{\partial \hat{y}_n}{\partial s_n}\frac{\partial s_n}{\partial s_k}\frac{\partial s_k}{\partial W}$

Tại `k = 0` ta có: 

> $\frac{\partial s_n}{\partial s_k} = \frac{\partial s_n}{\partial s_0} = \frac{\partial s_n}{\partial s_{n-1}}\frac{\partial s_{n-1}}{\partial s_{n-2}}...\frac{\partial s_2}{\partial s_1}\frac{\partial s_1}{\partial s_0}$

Trong quá trình thực hiện tính $\frac{\partial s_n}{\partial s_k}$, hàm activation $f$ là `tanh`, `sigmoid` có đạo hàm $f'$ < 1 nên gây ra hiện tượng Vanishing.

Vấn đề này còn được gọi là hiện tượng "quên" trong RNN.

##### 4.2.3 Vấn đề Exploding Gradient

Trong quá trình tính Backpropagation Algorithm cũng có thể gây ra hiện tượng Exploding ("bùng nổ").


### 5. Hướng khắc phục 

1. Thay đổi hàm kích hoạt.
2. Khởi tạo lại ma trận trọng số.
3. Thay đổi lại Node của RNN: sử dụng các mô hình tốt hơn như **Long short term memory (LSTM), Gated recurrent unit (GRU)**

### Tài liệu tham khảo
[1] [Recurrent Neural Networks Tutorial, Part 1 – Introduction to RNNs](http://www.wildml.com/2015/09/recurrent-neural-networks-tutorial-part-1-introduction-to-rnns/)

[2] [Google's Neural Machine Translation System](https://arxiv.org/abs/1609.08144)

[3] [End-to-End Deep Neural Network for Automatic Speech Recognition](https://cs224d.stanford.edu/reports/SongWilliam.pdf)

[4] [Deep Visual-Semantic Alignments for Generating Image Descriptions](https://cs.stanford.edu/people/karpathy/deepimagesent/)

[5] [Generating images from captions with attention](https://arxiv.org/pdf/1511.02793.pdf)

[6] [An Introduction to Recurrent Neural Networks](https://medium.com/explore-artificial-intelligence/an-introduction-to-recurrent-neural-networks-72c97bf0912)